# 🎬 Japanese to Chinese Subtitle Generator (Whisper Edition)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gatorbonita/translator/blob/main/whispermode/Japanese_to_Chinese_Subtitles_Colab.ipynb)

Generate high-quality Chinese subtitles for Japanese videos using:
- **Whisper** for transcription (excellent Japanese accuracy!)
- **Google Translate** for translation
- **FREE GPU** from Google Colab

---

## 📋 Quick Start

1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator: **GPU** → Save
2. **Run all cells**: Runtime → Run all (Ctrl+F9)
3. **Upload your video** when prompted
4. **Wait for processing** (~3-5 min for 30-min video)
5. **Download your .srt file**

---

## ⚙️ Step 1: Setup Environment

This will:
- Check if GPU is available
- Install required packages
- Takes ~2-3 minutes first time

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Check GPU availability
import torch
print("🔍 Checking GPU availability...")
if torch.cuda.is_available():
    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  No GPU detected, will use CPU (slower)")
    print("   Enable GPU: Runtime → Change runtime type → GPU")

print("\n📦 Installing dependencies...")
print("This may take 2-3 minutes on first run.\n")

# Install dependencies
!pip install -q moviepy faster-whisper google-cloud-translate python-dotenv loguru

print("\n✅ Setup complete!")

## 🔐 Step 2: Google Cloud Translation Setup

You need Google Cloud credentials **only for translation** (Whisper handles transcription locally).

### Option A: Upload Credentials File (Recommended)

Run the cell below and upload your `credentials.json` file when prompted.

### Option B: Skip Translation (Testing)

Set `SKIP_TRANSLATION = True` to skip translation and only test transcription.

---

**Don't have credentials?** [Quick Setup Guide](https://console.cloud.google.com):
1. Enable Cloud Translation API
2. Create Service Account → Download JSON key
3. Upload the JSON file here

In [ ]:
import os
from google.colab import files

# Set to True to skip translation (for testing transcription only)
SKIP_TRANSLATION = False

if not SKIP_TRANSLATION:
    print("📤 Please upload your Google Cloud credentials JSON file...")
    uploaded = files.upload()
    
    if uploaded:
        cred_filename = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = cred_filename
        print(f"✅ Credentials loaded: {cred_filename}")
    else:
        print("⚠️  No credentials uploaded. Will skip translation.")
        SKIP_TRANSLATION = True
else:
    print("⚠️  Translation disabled. Will only generate Japanese transcripts.")

## ⚙️ Step 3: Configuration

Adjust settings here:

- **WHISPER_MODEL**: `tiny`, `base`, `small`, `medium`, `large-v3`
  - `medium` = Best balance (recommended)
  - `large-v3` = Best quality (slower)
  - `small` = Faster, good quality

- **TARGET_LANGUAGE**: `zh-CN` (Simplified) or `zh-TW` (Traditional)

- **DEVICE**: `auto` (uses GPU if available), `cpu`, or `cuda`

In [ ]:
# Configuration
WHISPER_MODEL = 'medium'  # Options: tiny, base, small, medium, large-v3
TARGET_LANGUAGE = 'zh-CN'  # zh-CN = Simplified, zh-TW = Traditional
DEVICE = 'auto'  # auto = use GPU if available

print(f"⚙️  Configuration:")
print(f"   Whisper Model: {WHISPER_MODEL}")
print(f"   Target Language: {TARGET_LANGUAGE}")
print(f"   Device: {DEVICE}")
print(f"   Translation: {'Disabled' if SKIP_TRANSLATION else 'Enabled'}")

## 🔧 Step 4: Load Functions

Loading all the necessary functions...

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import List
from moviepy.editor import VideoFileClip, AudioFileClip
from faster_whisper import WhisperModel
from google.cloud import translate_v2 as translate
from google.oauth2 import service_account
import time
from datetime import datetime

@dataclass
class TranscriptSegment:
    """Represents a subtitle segment."""
    text: str
    start_time: float
    end_time: float
    confidence: float = 1.0

# Audio Extraction
def extract_audio(video_path, output_dir='/content'):
    """Extract audio from video."""
    print(f"\n🎵 Extracting audio from video...")
    video = VideoFileClip(video_path)
    
    if video.audio is None:
        raise Exception("Video has no audio track!")
    
    audio_path = f"{output_dir}/audio_temp.wav"
    video.audio.write_audiofile(
        audio_path,
        fps=16000,
        nbytes=2,
        codec='pcm_s16le',
        ffmpeg_params=['-ac', '1'],
        logger=None,
        verbose=False
    )
    
    duration = video.duration
    video.close()
    
    print(f"✅ Audio extracted: {duration:.1f} seconds")
    return audio_path, duration

# Whisper Transcription
def transcribe_with_whisper(audio_path, model_size='medium', device='auto'):
    """Transcribe audio with Whisper."""
    print(f"\n🎤 Transcribing with Whisper ({model_size} model)...")
    
    # Determine device and compute type
    if device == 'auto':
        if torch.cuda.is_available():
            device = 'cuda'
            compute_type = 'float16'
            print("   Using GPU acceleration")
        else:
            device = 'cpu'
            compute_type = 'int8'
            print("   Using CPU")
    elif device == 'cuda':
        compute_type = 'float16'
    else:
        compute_type = 'int8'
    
    # Load model
    print("   Loading Whisper model...")
    model = WhisperModel(model_size, device=device, compute_type=compute_type)
    
    # Transcribe
    print("   Transcribing... (this may take a few minutes)")
    segments, info = model.transcribe(
        audio_path,
        language='ja',
        beam_size=5,
        word_timestamps=True,
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=500)
    )
    
    print(f"   Detected language: {info.language} (confidence: {info.language_probability:.2%})")
    
    # Process segments
    transcript_segments = []
    for segment in segments:
        if segment.words:
            # Group words into subtitle-friendly segments
            word_segments = create_segments_from_words(segment.words)
            transcript_segments.extend(word_segments)
        else:
            transcript_segments.append(
                TranscriptSegment(
                    text=segment.text.strip(),
                    start_time=segment.start,
                    end_time=segment.end,
                    confidence=1.0
                )
            )
    
    print(f"✅ Transcription complete: {len(transcript_segments)} segments")
    return transcript_segments

def create_segments_from_words(words, max_duration=5.0, max_chars=80):
    """Group words into subtitle segments."""
    segments = []
    current_words = []
    current_start = None
    sentence_endings = {'。', '！', '？', '、'}
    
    for word in words:
        if current_start is None:
            current_start = word.start
        
        current_words.append(word.word)
        current_end = word.end
        duration = current_end - current_start
        text = ''.join(current_words).strip()
        
        should_finalize = False
        if duration >= max_duration or len(text) >= max_chars:
            should_finalize = True
        elif any(text.endswith(punct) for punct in sentence_endings):
            if len(text) > 10 or duration > 1.0:
                should_finalize = True
        
        if should_finalize:
            segments.append(TranscriptSegment(
                text=text,
                start_time=current_start,
                end_time=current_end,
                confidence=1.0
            ))
            current_words = []
            current_start = None
    
    if current_words:
        text = ''.join(current_words).strip()
        if text:
            segments.append(TranscriptSegment(
                text=text,
                start_time=current_start,
                end_time=current_end,
                confidence=1.0
            ))
    
    return segments

# Translation
def translate_segments(segments, target_language='zh-CN', batch_size=128):
    """Translate segments to Chinese."""
    if not segments:
        return []
    
    print(f"\n🌏 Translating to {target_language}...")
    
    # Initialize translator
    cred_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
    credentials = service_account.Credentials.from_service_account_file(cred_path)
    client = translate.Client(credentials=credentials)
    
    translated_segments = []
    
    # Process in batches
    for i in range(0, len(segments), batch_size):
        batch = segments[i:i + batch_size]
        texts = [seg.text for seg in batch]
        
        # Translate with retry
        for attempt in range(3):
            try:
                results = client.translate(texts, target_language=target_language, source_language='ja')
                if isinstance(results, dict):
                    translated_texts = [results['translatedText']]
                else:
                    translated_texts = [r['translatedText'] for r in results]
                break
            except Exception as e:
                if attempt == 2:
                    raise Exception(f"Translation failed: {e}")
                time.sleep(2 ** attempt)
        
        # Create translated segments
        for segment, translated_text in zip(batch, translated_texts):
            translated_segments.append(TranscriptSegment(
                text=translated_text,
                start_time=segment.start_time,
                end_time=segment.end_time,
                confidence=segment.confidence
            ))
        
        if len(segments) > batch_size:
            print(f"   Translated {min(i + batch_size, len(segments))}/{len(segments)} segments")
    
    print(f"✅ Translation complete: {len(translated_segments)} segments")
    return translated_segments

# SRT Generation
def generate_srt(segments, output_path):
    """Generate SRT subtitle file."""
    print(f"\n📝 Generating SRT file...")
    
    # Merge short segments
    merged = merge_short_segments(segments)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, segment in enumerate(merged, start=1):
            f.write(f"{i}\n")
            start_ts = format_timestamp(segment.start_time)
            end_ts = format_timestamp(segment.end_time)
            f.write(f"{start_ts} --> {end_ts}\n")
            f.write(f"{segment.text}\n\n")
    
    print(f"✅ SRT file created: {output_path}")
    return output_path

def format_timestamp(seconds):
    """Convert seconds to SRT timestamp format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millisecs = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millisecs:03d}"

def merge_short_segments(segments, min_duration=1.0, max_chars=80):
    """Merge segments that are too short."""
    if not segments:
        return []
    
    merged = []
    current = None
    
    for segment in segments:
        if current is None:
            current = TranscriptSegment(
                text=segment.text,
                start_time=segment.start_time,
                end_time=segment.end_time,
                confidence=segment.confidence
            )
            continue
        
        duration = current.end_time - current.start_time
        combined_text = current.text + " " + segment.text
        
        should_merge = (
            duration < min_duration or
            (len(combined_text) <= max_chars and segment.start_time - current.end_time < 1.0)
        )
        
        if should_merge:
            current.text = combined_text
            current.end_time = segment.end_time
        else:
            merged.append(current)
            current = TranscriptSegment(
                text=segment.text,
                start_time=segment.start_time,
                end_time=segment.end_time,
                confidence=segment.confidence
            )
    
    if current is not None:
        merged.append(current)
    
    return merged

print("✅ Functions loaded!")

## 📤 Step 5: Upload Video

Upload your Japanese video file. Supported formats:
- MP4, MKV, AVI, MOV, WebM, and more

**Note**: Large files (>100MB) may take time to upload. Consider using Google Drive for very large files.

In [ ]:
from google.colab import files

print("📤 Please upload your video file...")
uploaded = files.upload()

if uploaded:
    video_filename = list(uploaded.keys())[0]
    video_path = f"/content/{video_filename}"
    print(f"\n✅ Video uploaded: {video_filename}")
    print(f"   Size: {len(uploaded[video_filename]) / (1024*1024):.1f} MB")
else:
    raise Exception("No video file uploaded!")

## 🚀 Step 6: Process Video

This will:
1. Extract audio from video
2. Transcribe with Whisper (Japanese)
3. Translate to Chinese (if enabled)
4. Generate SRT subtitle file

**Estimated time** (30-min video with GPU):
- Audio extraction: ~30 seconds
- Whisper transcription: ~2-4 minutes
- Translation: ~10 seconds
- **Total: ~3-5 minutes**

In [ ]:
import time

start_time = time.time()

try:
    # Step 1: Extract audio
    audio_path, duration = extract_audio(video_path)
    
    # Step 2: Transcribe with Whisper
    japanese_segments = transcribe_with_whisper(
        audio_path,
        model_size=WHISPER_MODEL,
        device=DEVICE
    )
    
    # Step 3: Translate (if enabled)
    if SKIP_TRANSLATION:
        print("\n⚠️  Skipping translation (disabled)")
        final_segments = japanese_segments
        output_suffix = "_ja.srt"  # Japanese only
    else:
        final_segments = translate_segments(
            japanese_segments,
            target_language=TARGET_LANGUAGE
        )
        output_suffix = f"_{TARGET_LANGUAGE}.srt"
    
    # Step 4: Generate SRT
    video_name = Path(video_filename).stem
    output_path = f"/content/{video_name}{output_suffix}"
    generate_srt(final_segments, output_path)
    
    # Cleanup
    import os
    if os.path.exists(audio_path):
        os.remove(audio_path)
    
    # Summary
    elapsed = time.time() - start_time
    print("\n" + "="*60)
    print("🎉 SUBTITLE GENERATION COMPLETE!")
    print("="*60)
    print(f"Video duration: {duration:.1f} seconds ({duration/60:.1f} minutes)")
    print(f"Processing time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
    print(f"Segments: {len(final_segments)}")
    print(f"Model: {WHISPER_MODEL}")
    print(f"Output: {output_path}")
    print("="*60)
    
    SUBTITLE_FILE = output_path
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

## 📥 Step 7: Download Subtitle File

Download your generated subtitle file!

In [ ]:
from google.colab import files

try:
    if 'SUBTITLE_FILE' in globals() and os.path.exists(SUBTITLE_FILE):
        print(f"📥 Downloading: {SUBTITLE_FILE}")
        files.download(SUBTITLE_FILE)
        print("\n✅ Download started! Check your browser's download folder.")
        print("\n📺 To use:")
        print("   1. Open video in VLC player")
        print("   2. Subtitle → Add Subtitle File")
        print("   3. Select the downloaded .srt file")
    else:
        print("❌ No subtitle file found. Please run Step 6 first.")
except Exception as e:
    print(f"❌ Download error: {e}")

## 👀 (Optional) Preview Subtitles

Preview the first 10 subtitle entries:

In [ ]:
if 'SUBTITLE_FILE' in globals() and os.path.exists(SUBTITLE_FILE):
    print("📖 Preview of first 10 subtitles:\n")
    print("="*60)
    
    with open(SUBTITLE_FILE, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        preview_lines = []
        count = 0
        
        for line in lines:
            preview_lines.append(line)
            if line.strip() == '' and len(preview_lines) > 1:
                count += 1
                if count >= 10:
                    break
        
        print(''.join(preview_lines))
    
    print("="*60)
else:
    print("❌ No subtitle file found. Please run Step 6 first.")

---

## 💡 Tips & Troubleshooting

### For Better Results:
- ✅ **Use GPU**: Runtime → Change runtime type → GPU
- ✅ **Better quality**: Use `large-v3` model (slower)
- ✅ **Faster processing**: Use `small` or `base` model
- ✅ **Clear audio**: Remove background music if possible

### Common Issues:

**"Out of memory"**
- Use smaller model: `WHISPER_MODEL = 'small'`
- Runtime → Factory reset runtime

**"No GPU detected"**
- Runtime → Change runtime type → GPU → Save
- Restart runtime if needed

**"Translation error"**
- Check credentials file is uploaded
- Verify Translation API is enabled
- Set `SKIP_TRANSLATION = True` to test transcription only

**"Video upload fails"**
- For files >100MB, use Google Drive:
  ```python
  from google.colab import drive
  drive.mount('/content/drive')
  video_path = '/content/drive/MyDrive/your-video.mp4'
  ```

### Model Comparison:

| Model | VRAM | Speed | Quality |
|-------|------|-------|--------|
| tiny | 1GB | ⚡⚡⚡⚡⚡ | ⭐⭐ |
| base | 1GB | ⚡⚡⚡⚡ | ⭐⭐⭐ |
| small | 2GB | ⚡⚡⚡ | ⭐⭐⭐⭐ |
| medium | 5GB | ⚡⚡ | ⭐⭐⭐⭐⭐ |
| large-v3 | 10GB | ⚡ | ⭐⭐⭐⭐⭐ |

**Colab Free GPU**: ~15GB VRAM, can run up to `large-v3`

---

## 📚 Resources

- **GitHub Repository**: [gatorbonita/translator](https://github.com/gatorbonita/translator)
- **Whisper Edition Docs**: [whispermode/README.md](https://github.com/gatorbonita/translator/blob/main/whispermode/README.md)
- **Google Cloud Setup**: [Translation API Guide](https://cloud.google.com/translate/docs/setup)

---

<div align="center">

**🌟 Enjoying this tool? Star the repo on GitHub!**

Made with ❤️ using Whisper + Google Translate

</div>